In [1]:
from bs4 import BeautifulSoup
import pymongo
import requests
import re
import sys

In [3]:
# Create mongodb connection
from pymongo import MongoClient
connection = MongoClient('localhost', 27017)
#connection = Connection('localhost', 27017)

database = connection.housing_data

table = database.etuovi

#table.remove() to remove everything

cookie = {'org.springframework.mobile.device.site.CookieSitePreferenceRepository.SITE_PREFERENCE':'NORMAL','JSESSIONID':'183EE2F12C128B603790A0D5D1558022','evid_0006':'87ccc59d-4b80-489f-98cf-be92d6591fa6','evid_0006_ref':'false','adptseg_0006':'kv1001#a-kv1002#c-kv1003#d-kv1004#d-kv1005#c-kv1006#a:b-kv1008#c-kv1009#17-kv1010#b-kv1011#f-kv1012#c-kv1013#a-kv1016#1:11:13:2:7-kv1017#2-','adptset_0006':'1','evid_0006_set':'2','__utma':'82691985.304271103.1385626909.1385642039.1385658686.4','__utmb':'82691985.9.8.1385658723878','__utmc':'82691985','__utmz':'82691985.1385626909.1.1.utmcsr=kuluttaja.etuovi.com|utmccn=(referral)|utmcmd=referral|utmcct=/crometapp/product/realties/common/public/frontpage2/search_in_progress_to_vuokraovi.jsp','slbp_80':'2265258762.20480.0000','slbp_443':'2265258762.47873.0000'}

data_added = 0
data_existed = 0

In [44]:
start_page = 1
end_page = 3
Links = []
for page in range(start_page, end_page):
    print ('Parsing page:', str(page))
    html = requests.post('http://www.vuokraovi.com/vuokra-asunnot?page=' + str(page), cookies=cookie, data={'listorder':11})
    
    parsed_html = BeautifulSoup(html.text,'html.parser')                                                                                        
    #print(parsed_html.prettify())
    url_tag = parsed_html.find_all('a', {'class':'list-item-link'})
    for link in url_tag:
        address = link.get('href')
        if '/vuokra-asunto' in address:
            #print(address)
            Links.append(address)
print(Links)

    
#html_page = urllib2.urlopen("http://www.yourwebsite.com")
#soup = BeautifulSoup(html_page)
#for link in soup.findAll('a'):
#print link.get('href')

Parsing page: 1
Parsing page: 2
['/vuokra-asunto/espoo/ymmersta/kerrostalo/661148?entryPoint=fromSearch&rentalIndex=1', '/vuokra-asunto/espoo/ymmersta/kerrostalo/661147?entryPoint=fromSearch&rentalIndex=2', '/vuokra-asunto/espoo/ymmersta/kerrostalo/661146?entryPoint=fromSearch&rentalIndex=3', '/vuokra-asunto/lahti/keskusta/kerrostalo/661105?entryPoint=fromSearch&rentalIndex=4', '/vuokra-asunto/kuopio/rypysuo/kerrostalo/661145?entryPoint=fromSearch&rentalIndex=5', '/vuokra-asunto/vantaa/myyrmaki/kerrostalo/661144?entryPoint=fromSearch&rentalIndex=6', '/vuokra-asunto/helsinki/punavuori/kerrostalo/661143?entryPoint=fromSearch&rentalIndex=7', '/vuokra-asunto/helsinki/punavuori/kerrostalo/661142?entryPoint=fromSearch&rentalIndex=8', '/vuokra-asunto/helsinki/punavuori/kerrostalo/661141?entryPoint=fromSearch&rentalIndex=9', '/vuokra-asunto/helsinki/punavuori/kerrostalo/661140?entryPoint=fromSearch&rentalIndex=10', '/vuokra-asunto/helsinki/punavuori/kerrostalo/661139?entryPoint=fromSearch&rent

In [140]:
for page in Links:
    print(page)
    html = requests.post('http://www.vuokraovi.com' + page, cookies=cookie, data={'listorder':11})
    parsed_html = BeautifulSoup(html.text,'html.parser') 
    #print(parsed_html)
    test_tagth = parsed_html.find_all('th')
    test_tagtd = parsed_html.find_all('td')
    print('sijainti:')
    print(test_tagtd[0].find('span', {'itemprop':'addressLocality'}).get_text(strip = True)[:5])
    #print(test_tag[17])
    for i in range(0,len(test_tagth)):
        if 'Vakuus:' in test_tagth[i].get_text() \
        or 'Vuokra:' in test_tagth[i].get_text() \
        or 'Tyyppi:' in test_tagth[i].get_text() \
        or 'Huoneiden lukumäärä' in test_tagth[i].get_text() \
        or 'Rakennusvuosi:' in test_tagth[i].get_text() \
        or 'Yleiskunto:' in test_tagth[i].get_text() \
        or 'sallittu:' in test_tagth[i].get_text() \
        or 'pinta-ala:' in test_tagth[i].get_text():
            print(test_tagth[i].get_text(strip = True))
            print(test_tagtd[i].get_text(strip = True))
#So this is pretty good I think
    break



/vuokra-asunto/espoo/ymmersta/kerrostalo/661148?entryPoint=fromSearch&rentalIndex=1
sijainti:
02750
Tyyppi:
kerrostalo
Huoneiden lukumäärä:
Yksiö
Asuinpinta-ala:
38,5 m²
Kokonaispinta-ala:
38,5 m²
Yleiskunto:
hyvä
Vuokra:
770 €/kk
Vakuus:
1600 €
Lemmikit sallittu:
sallittu
Tupakointi sallittu:
ei sallittu


In [102]:
#This is for checking if the test_tagth[17] type of approach works

for page in Links:
    print(page)
    html = requests.post('http://www.vuokraovi.com' + page, cookies=cookie, data={'listorder':11})
    parsed_html = BeautifulSoup(html.text,'html.parser') 
    #print(parsed_html)
    test_tagth = parsed_html.find_all('th')
    test_tagtd = parsed_html.find_all('td')
    #print(test_tagtd[0].find('span', {'itemprop':'addressLocality'}).get_text(strip = True)[:5])
    print(test_tagth[17])


/vuokra-asunto/espoo/ymmersta/kerrostalo/661148?entryPoint=fromSearch&rentalIndex=1
<th>Vakuus:</th>
/vuokra-asunto/espoo/ymmersta/kerrostalo/661147?entryPoint=fromSearch&rentalIndex=2
<th>Vakuus:</th>
/vuokra-asunto/espoo/ymmersta/kerrostalo/661146?entryPoint=fromSearch&rentalIndex=3
<th>Vakuus:</th>
/vuokra-asunto/lahti/keskusta/kerrostalo/661105?entryPoint=fromSearch&rentalIndex=4
<th>
      Lemmikit sallittu:
    </th>
/vuokra-asunto/kuopio/rypysuo/kerrostalo/661145?entryPoint=fromSearch&rentalIndex=5
<th>Vakuus:</th>
/vuokra-asunto/vantaa/myyrmaki/kerrostalo/661144?entryPoint=fromSearch&rentalIndex=6
<th>Vakuus:</th>
/vuokra-asunto/helsinki/punavuori/kerrostalo/661143?entryPoint=fromSearch&rentalIndex=7
<th>Vakuus:</th>
/vuokra-asunto/helsinki/punavuori/kerrostalo/661142?entryPoint=fromSearch&rentalIndex=8
<th>Vakuus:</th>
/vuokra-asunto/helsinki/punavuori/kerrostalo/661141?entryPoint=fromSearch&rentalIndex=9
<th>Vakuus:</th>
/vuokra-asunto/helsinki/punavuori/kerrostalo/661140?ent

KeyboardInterrupt: 